In [7]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler


In [3]:
df = pd.read_excel(r'C:\Users\Anderson\Desktop\we_fn_use_c_marketing_customer_value_analysis.xlsx')
df

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2.763519e+09,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,3.848111e+08,Two-Door Car,Medsize
1,QZ44356,Arizona,6.979536e+09,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1.131465e+09,Four-Door Car,Medsize
2,AI49188,Nevada,1.288743e+09,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,5.664722e+08,Two-Door Car,Medsize
3,WW63253,California,7.645862e+09,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,5.298813e+08,SUV,Medsize
4,HB64268,Washington,2.813693e+09,No,Basic,Bachelor,2011-03-02 00:00:00,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,1.381309e+08,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,2.340599e+09,No,Basic,Bachelor,2011-10-02 00:00:00,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,1.982348e+08,Four-Door Car,Medsize
9130,PK87824,California,3.096511e+09,Yes,Extended,College,2011-12-02 00:00:00,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,3.792000e+02,Four-Door Car,Medsize
9131,TD14365,California,8.163890e+09,No,Extended,Bachelor,2011-06-02 00:00:00,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,7.907850e+08,Four-Door Car,Medsize
9132,UP19263,California,7.524442e+09,No,Extended,College,2011-03-02 00:00:00,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,6.912000e+02,Four-Door Car,Large


In [10]:
print(df.isnull().sum())

Customer                         0
State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
Effective To Date                0
EmploymentStatus                 0
Gender                           0
Income                           0
Location Code                    0
Marital Status                   0
Monthly Premium Auto             0
Months Since Last Claim          0
Months Since Policy Inception    0
Number of Open Complaints        0
Number of Policies               0
Policy Type                      0
Policy                           0
Renew Offer Type                 0
Sales Channel                    0
Total Claim Amount               0
Vehicle Class                    0
Vehicle Size                     0
dtype: int64


In [ ]:
#We will start with removing outliers. So far, we have discussed different methods to remove outliers. 
#Use the one you feel more comfortable with, define a function for that. 
#Use the function to remove the outliers and apply it to the dataframe.

In [5]:
def remove_outliers(df, threshold=3):
    z_scores = np.abs(zscore(df.select_dtypes(include=[np.number])))
    
    df_clean = df[(z_scores < threshold).all(axis=1)]
    
    return df_clean

data_clean = remove_outliers(df)

In [ ]:
#Create a copy of the dataframe for the data wrangling.

In [6]:
data_wrangle = data_clean.copy()

In [ ]:
#Normalize the continuous variables. You can use any one method you want.

In [8]:
continuous_cols = data_wrangle.select_dtypes(include=[np.number]).columns
scaler = MinMaxScaler()
data_wrangle[continuous_cols] = scaler.fit_transform(data_wrangle[continuous_cols])

In [ ]:
#4.Encode the categorical variables

In [13]:
data_wrangle['Coverage'] = data_wrangle['Coverage'].map({"Basic": 0, "Extended": 1, "Premium": 2})
data_wrangle['EmploymentStatus'] = data_wrangle['EmploymentStatus'].map({"Unemployed": 0, "Employed": 1, "Retired": 2})
data_wrangle['Location Code'] = data_wrangle['Location Code'].map({"Rural": 0, "Suburban": 1, "Urban": 2})
data_wrangle['Vehicle Size'] = data_wrangle['Vehicle Size'].map({"Small": 0, "Medsize": 1, "Large": 2})

data_wrangle = pd.get_dummies(data_wrangle, columns=[
    'State', 'Marital Status', 'Policy Type', 'Policy', 
    'Renew Offer Type', 'Customer', 'Sales Channel', 'Vehicle Class'
])

In [ ]:
#5.The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [17]:
data_wrangle['Effective To Date'] = pd.to_datetime(data_wrangle['Effective To Date'])

C:\Users\Anderson\AppData\Local\Temp\ipykernel_6612\2511329864.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_wrangle['Effective To Date'] = pd.to_datetime(data_wrangle['Effective To Date'])


In [19]:
data_wrangle['day'] = data_wrangle['Effective To Date'].dt.day
data_wrangle['week'] = data_wrangle['Effective To Date'].dt.isocalendar().week
data_wrangle['month'] = data_wrangle['Effective To Date'].dt.month


# Drop the original 'time' column if not needed
data_wrangle.drop('Effective To Date', axis=1, inplace=True)

In [ ]:
#6.Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [23]:
for column in data_wrangle.columns:
    print(f"{column}: {data_wrangle[column].dtype}")

Customer Lifetime Value: float64
Response: object
Coverage: float64
Education: object
EmploymentStatus: float64
Gender: object
Income: float64
Location Code: float64
Monthly Premium Auto: float64
Months Since Last Claim: float64
Months Since Policy Inception: float64
Number of Open Complaints: float64
Number of Policies: float64
Total Claim Amount: float64
Vehicle Size: float64
State_Arizona: bool
State_California: bool
State_Nevada: bool
State_Oregon: bool
State_Washington: bool
Marital Status_Divorced: bool
Marital Status_Married: bool
Marital Status_Single: bool
Policy Type_Corporate Auto: bool
Policy Type_Personal Auto: bool
Policy Type_Special Auto: bool
Policy_Corporate L1: bool
Policy_Corporate L2: bool
Policy_Corporate L3: bool
Policy_Personal L1: bool
Policy_Personal L2: bool
Policy_Personal L3: bool
Policy_Special L1: bool
Policy_Special L2: bool
Policy_Special L3: bool
Renew Offer Type_Offer1: bool
Renew Offer Type_Offer2: bool
Renew Offer Type_Offer3: bool
Renew Offer Type_